In [1]:
import os
from together import Together
from dotenv import load_dotenv
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
import nest_asyncio
import glob
import requests
import re
from llama_index.llms.together import TogetherLLM
from llama_index.core.llms import ChatMessage


c:\Users\moh00\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\moh00\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\moh00\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
load_dotenv()
nest_asyncio.apply()

In [3]:
def webscrape_url(url):
    url = url.replace("https://", "")
    response = requests.get("https://r.jina.ai/" + url)
    return response.text

In [4]:
data1 = webscrape_url("https://rakuten.wd1.myworkdayjobs.com/RakutenAmericas/job/Toronto-Canada/Intern---Software-Engineer_1020526-2?source=LinkedIn")
print(data1)

Title: Intern - Software Engineer

URL Source: http://rakuten.wd1.myworkdayjobs.com/RakutenAmericas/job/Toronto-Canada/Intern---Software-Engineer_1020526-2?source=LinkedIn

Markdown Content:
**Job Description:**

The Intern Experience:

At Rakuten, we pride ourselves on giving interns a genuine, immersive experience that has a direct impact on company objectives. You will do meaningful work, and with us, you’ll be a true member of the team. We are also invested in your growth as an individual and professional. That’s why you can expect curated events such as professional development workshops, interactions with Rakuten leadership, and relationship building opportunities with other interns and professionals at the company.

**You can visit** [**Rakuten.us**](http://rakuten.us/)**,** [**our Muse profile**](https://www.themuse.com/profiles/rakutenamericas/team/internships)**,** **or listen to our feature on** [**The Internship Show Podcast**](https://podcasters.spotify.com/pod/show/the-in

In [5]:
parser = LlamaParse(
    result_type="markdown"  # "markdown" and "text" are available
)

file_extractor = {".pdf": parser}


In [6]:
document_CV = SimpleDirectoryReader(input_files=glob.glob('assets\\CV_M_ALI_JULY.pdf'), file_extractor=file_extractor).load_data()
document_linkedin = SimpleDirectoryReader(input_files=glob.glob('assets\\Profile.pdf'), file_extractor=file_extractor).load_data()

Started parsing the file under job_id cac11eca-76f5-46fa-b5ad-153d9d285ae8
Started parsing the file under job_id cac11eca-44d7-4cf9-a114-30a4b46621b6


In [7]:
full_text_linkedin = '\n\n'.join([page.text for page in document_linkedin])
full_text_CV = '\n\n'.join([page.text for page in document_CV])

In [8]:
llm = TogetherLLM(model="meta-llama/Meta-Llama-3-70B-Instruct-Turbo",
            api_key=os.environ.get('TOGETHER_API_KEY'),max_tokens=4096,temperature=0.1,
    repetition_penalty=1,
    stop=["<|eot_id|>"],
    stream=False)

In [24]:
CV = full_text_CV
job_details = data1
linkedin = full_text_linkedin


messages = [
    ChatMessage(
        role="system", content="You are an AI assitance that extract key information from user CV, lindedin profile and job post and only output the results in multiple unique bullet points"
    ),
    ChatMessage(role="user", content=f"""
CV : {CV}
Linkedin: {linkedin}
Job details: {job_details}
"""),
]

resp = llm.chat(messages)
print(resp.message.content)

Here are the key information extracted from the CV, LinkedIn profile, and job post in multiple unique bullet points:

**Summary**

* Mohamed A. Ali is a Machine Learning Engineer with a strong foundation in problem-solving and analytical background.
* He has expertise in delivering significant results, including cost-saving mobile OCR models and improved risk assessment algorithms.
* He excels in collaborative environments and has integrated multiple models for real-time inference and co-developed innovative AI solutions.

**Experience**

* Machine Learning Engineer at Blnk.ai (Mar 2023 - Present)
	+ Created a RAG system using open source LLMs for company’s technical documentation.
	+ Built a machine learning pipeline using Whisper (OpenAI LLM) fine-tuned to transcribe Arabic call center conversations.
	+ Trained and deployed in-house efficient quantized OCR deep learning models on mobile devices.
	+ Enhanced delinquency and risk assessment algorithms, improving accuracy by 20%.
* Sess

In [11]:
context = resp.message.content
question = "why do you think you fit the role?"

messages = [
    ChatMessage(
        role="system", content=f"""You are an AI assitance that answer job related questions based on 
        context provided which include key informtation of user CV, linkedin profile and job. Answer in a formal way and
        don't mention the source of your information.
        context = {context}
        """
    ),
    ChatMessage(role="user", content=f"""qestion: {question}
                Answer: """),
]

resp = llm.chat(messages)
print(resp.message.content)

I believe I fit the role of Intern - Software Engineer at Rakuten for several reasons. Firstly, I possess strong software development abilities in languages such as Python, Java, C, and C++, which aligns with the job requirements. My experience in developing and deploying machine learning models demonstrates my ability to write efficient and effective code.

Additionally, my experience as a Session Lead at Udacity has honed my ability to communicate complex technical concepts effectively and work collaboratively with diverse stakeholders, which is essential for the role. Furthermore, my solid understanding of data science and miscellaneous technologies, including machine learning, deep learning, and NLP, enables me to analyze data and develop metrics that drive business decisions.

Lastly, my strong communication skills, developed through my experience as a Machine Learning Engineer and Session Lead, allow me to work effectively with cross-functional teams and stakeholders. Overall, my

In [25]:
context = resp.message.content

messages = f"""You are an AI assitance that writes a cover letter based on 
        context provided which include key informtation of user CV, linkedin profile and
        job details.
        Answer in a formal and clear way and don't mention source of information.
        Use the name in the CV for the cover letter signing off.
        context = {context}
        cover letter:

        """

resp = llm.complete(messages)
print(resp.text)

Dear Hiring Manager,

I am excited to apply for the Intern - Software Engineer position at Rakuten, where I can leverage my skills and experience in machine learning and software development to drive business growth and innovation.

As a Machine Learning Engineer with a strong foundation in problem-solving and analytical background, I am confident in my ability to develop multiple approaches to a solution and deploy them in an A/B test to customers. My expertise in delivering significant results, including cost-saving mobile OCR models and improved risk assessment algorithms, demonstrates my capability to analyze, develop, and recommend metrics to drive business decisions.

In my current role as a Machine Learning Engineer at Blnk.ai, I have successfully created a RAG system using open source LLMs for company’s technical documentation and built a machine learning pipeline using Whisper (OpenAI LLM) fine-tuned to transcribe Arabic call center conversations. I have also trained and deplo